## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. Make sure you fill in any place that says `# BEGIN CODE HERE #END CODE HERE`. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run" (denoted by a play symbol). Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). 

 **What you need to remember:**

- Run your cells using SHIFT+ENTER (or "Run cell")
- Write code in the designated areas using Python 3 only
- Do not modify the code outside of the designated areas
- In some cases you will also need to explain the results. There will also be designated areas for that. 

Fill in your **NAME** and **AEM** below:

In [ ]:
NAME = "Eleni Mandana"
AEM = "3580"

---

# Assignment 3 - Ensemble Methods #

Welcome to your third assignment. This exercise will test your understanding on Ensemble Methods.

In [1]:
# Always run this cell
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier, StackingClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, accuracy_score 
from sklearn import model_selection 

# USE THE FOLLOWING RANDOM STATE FOR YOUR CODE
RANDOM_STATE = 42

## Download the Dataset ##
Download the dataset using the following cell or from this [link](https://github.com/sakrifor/public/tree/master/machine_learning_course/EnsembleDataset) and put the files in the same folder as the .ipynb file. 
In this assignment you are going to work with a dataset originated from the [ImageCLEFmed: The Medical Task 2016](https://www.imageclef.org/2016/medical) and the **Compound figure detection** subtask. The goal of this subtask is to identify whether a figure is a compound figure (one image consists of more than one figure) or not. The train dataset consits of 4197 examples/figures and each figure has 4096 features which were extracted using a deep neural network. The *CLASS* column represents the class of each example where 1 is a compoung figure and 0 is not. 


In [2]:
import urllib.request
url_train = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/train_set.csv'
filename_train = 'train_set.csv'
urllib.request.urlretrieve(url_train, filename_train)
url_test = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/test_set_noclass.csv'
filename_test = 'test_set_noclass.csv'
urllib.request.urlretrieve(url_test, filename_test)

In [ ]:
# Run this cell to load the data
train_set = pd.read_csv("train_set.csv").sample(frac=1).reset_index(drop=True)
train_set.head()
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values

## 1.0 Testing different ensemble methods ##
In this part of the assignment you are asked to create and test different ensemble methods using the train_set.csv dataset. You should use **10-fold cross validation** for your tests and report the average f-measure weighted and balanced accuracy of your models. You can use [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) and select both metrics to be measured during the evaluation. Otherwise, you can use [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold).

### !!! Use n_jobs=-1 where is posibble to use all the cores of a machine for running your tests ###

### 1.1 Voting ###
Create a voting classifier which uses three **simple** estimators/classifiers. Test both soft and hard voting and choose the best one. Consider as simple estimators the following:


*   Decision Trees
*   Linear Models
*   Probabilistic Models (Naive Bayes)
*   KNN Models  

In [32]:
# BEGIN CODE HERE

# RUNNING TIME : 2min

cls1 = DecisionTreeClassifier(criterion='gini', max_depth=12, random_state=RANDOM_STATE) # Classifier #1 
cls2 = LogisticRegression(class_weight='balanced', random_state=RANDOM_STATE) # Classifier #2 
cls3 = KNeighborsClassifier(n_neighbors=10, n_jobs=-1) # Classifier #3
soft_vcls = VotingClassifier(estimators=[
                            ('dt', cls1), ('lr', cls2), ('knn', cls3)], voting='soft', n_jobs=-1) # Voting Classifier
hard_vcls = VotingClassifier(estimators=[
                            ('dt', cls1), ('lr', cls2), ('knn', cls3)], voting='hard', n_jobs=-1, ) # Voting Classifier

soft_vcls.fit(X, y)
svlcs_predict = model_selection.cross_val_predict(soft_vcls, X, y, cv=10) # Cls pred

s_avg_fmeasure = np.mean(f1_score(y, svlcs_predict, average='weighted')) # The average f-measure
s_avg_accuracy = np.mean(accuracy_score(y, svlcs_predict)) # The average accuracy

hard_vcls.fit(X, y)
hvlcs_predict = model_selection.cross_val_predict(hard_vcls, X, y, cv=10) # Cls pred

h_avg_fmeasure = np.mean(f1_score(y, hvlcs_predict, average='weighted')) # The average f-measure
h_avg_accuracy = np.mean(accuracy_score(y, hvlcs_predict)) # The average accuracy
#END CODE HERE

In [33]:
print("Classifier:")
print(soft_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(s_avg_fmeasure,4), round(s_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(max_depth=12,
                                                     random_state=42)),
                             ('lr',
                              LogisticRegression(class_weight='balanced',
                                                 random_state=42)),
                             ('knn',
                              KNeighborsClassifier(n_jobs=-1, n_neighbors=10))],
                 n_jobs=-1, voting='soft')
F1 Weighted-Score: 0.8268 & Balanced Accuracy: 0.8275


You should achive above 82% (Soft Voting Classifier)

In [34]:
print("Classifier:")
print(hard_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(h_avg_fmeasure,4), round(h_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(max_depth=12,
                                                     random_state=42)),
                             ('lr',
                              LogisticRegression(class_weight='balanced',
                                                 random_state=42)),
                             ('knn',
                              KNeighborsClassifier(n_jobs=-1, n_neighbors=10))],
                 n_jobs=-1)
F1 Weighted-Score: 0.8293 & Balanced Accuracy: 0.8294


You should achieve above 80% in both! (Hard Voting Classifier)

### 1.2 Stacking ###
Create a stacking classifier which uses two more complex estimators. Try different simple classifiers (like the ones mentioned before) for the combination of the initial estimators. Report your results in the following cell.

Consider as complex estimators the following:

*   Random Forest
*   SVM
*   Gradient Boosting
*   MLP




In [7]:
# BEGIN CODE HERE

cls1 = GradientBoostingClassifier(random_state=RANDOM_STATE) # Classifier #1 
cls2 = RandomForestClassifier(n_estimators=150, random_state=RANDOM_STATE, n_jobs=-1) # Classifier #2 
cls3 = MLPClassifier(random_state=RANDOM_STATE) #cls3 = DecisionTreeClassifier(max_depth=3, random_state=RANDOM_STATE) # Classifier #3 (Optional) max_d = 5
scls = StackingClassifier(estimators=[
                            ('gb', cls1), ('rf', cls2), ('mlp', cls3)], final_estimator=LogisticRegression(), n_jobs=-1) # Stacking Classifier
scls.fit(X, y)

scoring = {'f_measure' : 'f1_weighted', 'acc' : 'balanced_accuracy'}
scls_predict = model_selection.cross_validate(scls, X, y, cv=10, scoring=scoring)

avg_fmeasure = np.mean(scls_predict['test_f_measure']) # The average f-measure
avg_accuracy = np.mean(scls_predict['test_acc']) # The average accuracy
#END CODE HERE

In [8]:
print("Classifier:")
print(scls)
print("F1 Weighted Score: {} & Balanced Accuracy: {}".format(round(avg_fmeasure,4), round(avg_accuracy,4)))

Classifier:
StackingClassifier(estimators=[('gb',
                                GradientBoostingClassifier(random_state=42)),
                               ('rf',
                                RandomForestClassifier(n_estimators=150,
                                                       n_jobs=-1,
                                                       random_state=42)),
                               ('mlp', MLPClassifier(random_state=42))],
                   final_estimator=LogisticRegression(), n_jobs=-1)
F1 Weighted Score: 0.8617 & Balanced Accuracy: 0.855


You should achieve above 85% in both

## 2.0 Randomization ##

**2.1** You are asked to create three ensembles of decision trees where each one uses a different method for producing homogeneous ensembles. Compare them with a simple decision tree classifier and report your results in the dictionaries (dict) below using as key the given name of your classifier and as value the f1_weighted/balanced_accuracy score. The dictionaries should contain four different elements.  

In [5]:
# BEGIN CODE HERE
# TODO fix these
tree = DecisionTreeClassifier()

ens1 = GradientBoostingClassifier(random_state=RANDOM_STATE)
ens2 = BaggingClassifier(random_state=RANDOM_STATE)
ens3 = AdaBoostClassifier(random_state=RANDOM_STATE)

ens1.fit(X, y)
ens1_pred = model_selection.cross_val_predict(ens1, X, y, cv=10) #ens1.predict(X)

ens2.fit(X, y)
ens2_pred = model_selection.cross_val_predict(ens2, X, y, cv=10) #ens2.predict(X)

ens3.fit(X, y)
ens3_pred = model_selection.cross_val_predict(ens3, X, y, cv=10) #ens3.predict(X)

tree.fit(X, y)
tree_pred = model_selection.cross_val_predict(tree, X, y, cv=10) #tree.predict(X)

f_measures = dict([
    (" Simple Desicion", f1_score(tree_pred, y, average='weighted')),
    (" Enseble with Gradient Boosting Classifier", f1_score(ens1_pred, y, average='weighted')),
    (" Ensemble with Bagging Classifier", f1_score(ens2_pred, y, average='weighted')),
    (" Enseble with Ada Boosting", f1_score(ens3_pred, y, average='weighted'))
])
accuracies = dict([
    (" Simple Desicion", accuracy_score(tree_pred, y)),
    (" Enseble with Gradient Boosting Classifier", accuracy_score(ens1_pred, y)),
    (" Ensemble with Bagging Classifier", accuracy_score(ens2_pred, y)),
    (" Enseble with Ada Boosting Classifier", accuracy_score(ens3_pred, y))
])
# Example f_measures = {'Simple Decision': 0.8551, 'Ensemble with random ...': 0.92, ...}

#END CODE HERE

In [6]:
print(ens1)
print(ens2)
print(ens3)
print(tree)
for name,score in f_measures.items():
    print("Classifier:{} -  F1 Weighted:{}".format(name,round(score,4)))
for name,score in accuracies.items():
    print("Classifier:{} -  BalancedAccuracy:{}".format(name,round(score,4)))

RandomForestClassifier(criterion='entropy', random_state=42)
RandomForestClassifier(n_estimators=110, random_state=42)
RandomForestClassifier(n_estimators=150, random_state=42)
DecisionTreeClassifier()
Classifier: Simple Desicion -  F1 Weighted:0.699
Classifier: Enseble with Gradient Boosting Classifier -  F1 Weighted:0.814
Classifier: Ensemble with Bagging Classifier -  F1 Weighted:0.8127
Classifier: Enseble with Ada Boosting -  F1 Weighted:0.8135
Classifier: Simple Desicion -  BalancedAccuracy:0.699
Classifier: Enseble with Gradient Boosting Classifier -  BalancedAccuracy:0.8103
Classifier: Ensemble with Bagging Classifier -  BalancedAccuracy:0.8093
Classifier: Enseble with Ada Boosting Classifier -  BalancedAccuracy:0.8101


**2.2** Describe your classifiers and your results.

Έχω επιλέξει τη χρήση ενός απλού Desicion Tree Classifier, και οι υπόλοιποι ensembles είναι οι Gradient Boosting, Bagging και Ada Boosting Classifier. Δεν χρειάστηκε να πειράξω κάποια παράμετρο γιατί τα σκορς τους ήταν ΟΚ εξ αρχής. Μέσο όρο οι ensembles είναι πολύ κοντά στο 80% και στις 2 μετρικές ενώ ο απλός DTC έχει εμφανώς χαμηλότερα σκορς από όλους. Επίσης οι προβλέψεις τους προέκυψαν από cross validation.

**2.3** Increasing the number of estimators in a bagging classifier can drastically increase the training time of a classifier. Is there any solution to this problem? Can the same solution be applied to boosting classifiers?


Στους bagging classifiers τα μοντέλα του συνόλου μπορούν τόσο να εκπαιδευτούν όσο και να χρησιμοποιηθούν για προβλέψεις ανεξάρτητα το ένα από το άλλο. Επομένως, θα μπορούσε να γίνει χρήση πολλαπλών υπολογιστικών πόρων για την επιτάχυνση των διαδικασιών εκπαίδευσης. Δεδομένου ότι τα μοντέλα των boosting classifiers εκπαιδεύονται με τέτοιο τρόπο που απαιτεί να έχουν τα αποτελέσματα του τελευταίου που εκπαιδεύτηκε, μία λύση που αφορά παραλληλισμό, δεν θα μπορέσει να δουλέψει σε μία γραμμική τεχνική εκπαίδευσης. Επομένως η απάντηση είναι όχι.

## 3.0 Creating the best classifier ##

**3.1** In this part of the assignment you are asked to train the best possible ensemble! Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure (weighted) & balanced accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code. Can you achieve a balanced accuracy over 83-84%?

In [3]:
# BEGIN CODE HERE

'''
from sklearn.model_selection import KFold, cross_val_score
def train_model(clf, x_train, y_train, n_folds):
    cv = KFold(n_splits = n_folds, shuffle=True, random_state=RANDOM_STATE)
    scores = cross_val_score(clf, x_train, y_train, scoring='balanced_accuracy', cv=cv, n_jobs=-1)
    return np.mean(scores)

rf_params = [
    {
    'n_estimators': [100, 150, 200],
    'max_depth': [2, 3]
    }
]


clf1 = RandomForestClassifier(criterion='gini', n_estimators=150, n_jobs=-1)
rfc_score = train_model(clf1, X, y, 10)

grid_search = GridSearchCV(clf1, rf_params, cv=10, scoring='balanced_accuracy')

grid_search.fit(X, y)
y_predict =  grid_search.predict(X)

print("Best params:")
print(grid_search.best_params_)
'''
cls1 = RandomForestClassifier(criterion='gini', n_jobs=-1, random_state=RANDOM_STATE)
cls2 = AdaBoostClassifier(random_state=RANDOM_STATE)
cls3 = MLPClassifier(random_state=RANDOM_STATE)

best_cls = StackingClassifier(estimators=[
                            ('rf', cls1), ('ab', cls2), ('mlp', cls3)], final_estimator=LogisticRegression(random_state=RANDOM_STATE), n_jobs=-1)
best_cls.fit(X, y)

scoring = {'f_measure' : 'f1_weighted', 'acc' : 'balanced_accuracy'}
prediction = model_selection.cross_validate(scls, X, y, cv=10, scoring=scoring)

best_fmeasure = np.mean(scls_predict['test_f_measure'])
best_accuracy = np.mean(scls_predict['test_acc'])
#END CODE HERE

0.859543963501341
0.8526601786849513


In [4]:
# print(grid_search.best_score_)
print("Classifier:")
#print(best_cls)
print("F1 Weighted-Score:{} & Balanced Accuracy:{}".format(best_fmeasure, best_accuracy))

Classifier:
F1 Weighted-Score: & Balanced Accuracy:


**3.2** Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure & accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code.

Από τα παραπάνω ερωτήματα, ξεχώρισε η χρήση του MLP οπότε αποφάσισα να τον χρησιμοποιήσω, αναγνωρίζω βέβεαι πως η επιτυχία του οφείλεται στο ότι ειναι νευρωνικό μοντέλο. Ήθελα να υπάρχει μία τυχαιότητα και για αυτό επέλεξα τη χρήση του Random Forest, αλλά και να βελτιωθεί το μοντέλο που με οδήγησε στη χρήση του AdaBoost. Θεώρησα ότι ο συνδιασμός τους θα έχει καλά αποτελέσματα και όντως έχει. O final estimator προτίμησα να μείνει κάτι απλό λόγω χρόνου κυρίως. Με τη πρώτη προσπάθεια, με μία μικρή αύξηση στους estimators του AdaBoost πέτυχα balanced accuracy 83.8%. Στη δοκιμή να παραμείνουν στους default (50) το αποτέλεσμα ήταν καλύτερο και έφτασε balanced accuracy 85.2%. Θα χρησιμοποιηθεί το ίδιο μοντέλο και στο επόμενο κομμάτι της εργασίας, δεδομένου του καλού σκορ του και λόγω χρόνου εκτέλεσης. Δυστυχώς δεν υπάρχει η δυνατότητα για κάποιο grid search.

**3.3** Create a classifier that is going to be used in production - in a live system. Use the *test_set_noclass.csv* to make predictions. Store the predictions in a list.  

In [5]:
# BEGIN CODE HERE
cls = best_cls
#END CODE HERE
test_set = pd.read_csv("test_set_noclass.csv")
predictions = cls.predict(test_set)

AttributeError: 'str' object has no attribute 'predict'

LEAVE HERE ANY COMMENTS ABOUT YOUR CLASSIFIER

#### This following cell will not be executed. The test_set.csv with the classes will be made available after the deadline and this cell is for testing purposes!!! Do not modify it! ###

In [ ]:
if False:
  from sklearn.metrics import f1_score, balanced_accuracy_score
  final_test_set = pd.read_csv('test_set.csv')
  ground_truth = final_test_set['CLASS']
  print("Balanced Accuracy: {}".format(balanced_accuracy_score(predictions, ground_truth)))
  print("F1 Weighted-Score: {}".format(f1_score(predictions, ground_truth, average='weighted')))

Both should aim above 85%!